In [65]:
# from opik.integrations.langchain import OpikTracer

# from opik import track

# opik_tracer = OpikTracer(project_name="langchain-examples")

from opik import configure 
from opik.integrations.langchain import OpikTracer 

configure() 
opik_tracer = OpikTracer(thread_id='user1') 

OPIK: Existing Opik clients will not use updated values for "url", "api_key", "workspace".
OPIK: Opik is already configured. You can check the settings by viewing the config file at /Users/aarsh/.opik.config
OPIK: Configuration completed successfully. Traces will be logged to 'Default Project' project. To change the destination project, see: https://www.comet.com/docs/opik/tracing/log_traces#configuring-the-project-name


In [66]:
import os
from typing import List, Dict
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI 
from langchain.tools import tool
from sqlalchemy import create_engine, Column, Integer, String, Text
from sqlalchemy.orm import sessionmaker, declarative_base
from pprint import pprint
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [67]:
from datetime import datetime
from zoneinfo import ZoneInfo
# from langchain_core.tools import tool


In [68]:
llm = ChatOpenAI(base_url=os.getenv('BASE_URL'),
                 api_key=os.getenv('API_KEY'),
                 model=os.getenv('MODEL_NAME'), 
                 temperature=0.1)

In [69]:
# --- 1. Database Persistence (SQLAlchemy) ---
Base = declarative_base()

class ResolutionEntry(Base):
    __tablename__ = 'resolution_plans'
    id = Column(Integer, primary_key=True)
    resolution = Column(String(500))
    plan_json = Column(Text)

engine = create_engine('sqlite:///stride_resolutions.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)

In [70]:
@tool
def save_resolution_to_db(resolution: str, plan_summary: str) -> str:
    """Saves the resolution and the generated milestones to the database."""
    session = Session()
    try:
        new_entry = ResolutionEntry(resolution=resolution, plan_json=plan_summary)
        session.add(new_entry)
        session.commit()
        return "Successfully saved to database."
    except Exception as e:
        session.rollback()
        return f"Error saving to database: {str(e)}"
    finally:
        session.close()

In [71]:
@tool
def current_time(tz: str = "Asia/Kolkata") -> str:
    """Get the current date/time in ISO format for a given IANA timezone."""
    return datetime.now(ZoneInfo(tz)).isoformat()


In [72]:
# --- 2. Structured Output Schema ---
class Milestone(BaseModel):
    period: int = Field(description="month number or week number")
    task: str = Field(description="The specific actionable milestone or habit")
    focus: str = Field(description="The primary objective for this unit")

class CalendarPlan(BaseModel):
    monthly_milestones: List[Milestone] = Field(description="monthly high-level goals")
    weekly_breakdown: Dict[int, List[Milestone]] = Field(description="each week focus areas for the all months where key is month and value is list of weekly milestones")
    daily_habits: List[str] = Field(description="Consistent daily actions to maintain momentum")

In [73]:
class FeasibilityVerdict(BaseModel):
    feasible: bool
    reason: str


In [74]:
def check_resolution_feasibility(user_resolution: str):
    agent = create_agent(
        model=llm,
        system_prompt = """
                        You assess whether a user’s goal is realistically achievable by an average human within the stated timeframe.
                        
                        Tool use:
                        - If the user gives a deadline date/time (e.g., "by tomorrow", "by Jan 31", "in 3 days"), you MUST call `current_time`
                          to anchor relative time (today/tomorrow/next week) and interpret the timeframe correctly.
                        - If the user gives an explicit duration (e.g., "in 30 days", "in 1 month"), you may skip calling the tool.
                        
                        Rules:
                        - If the goal or timeframe is missing/unclear, set feasible=false and explain what timeframe/info is needed.
                        - Assume typical starting skill and typical constraints unless the user provides specifics.
                        - Consider basic limits: time available, learning/skill acquisition rates, safety, and practicality.
                        - Be conservative: if it’s doubtful, set feasible=false.
                        - Provide a short, concrete reason (1–3 sentences). Do not provide a plan.
                        - Output MUST match the provided schema exactly.
                        """,
        tools=[current_time],
        response_format=FeasibilityVerdict  # Ensures the agent returns the exact Pydantic schema
    )
    result = agent.invoke({
        "messages": [
            {"role": "user", "content": f"My resolution is: {user_resolution}"}
        ]},
        config={"callbacks": [opik_tracer]}
                         )
    return result["structured_response"]

In [75]:
# --- 3. Agent Configuration ---
def run_resolution_agent(user_resolution: str):
    # Initialize the model (v1 uses improved content blocks)

    # create_agent is the v1 standard for building ReAct-style agents
    agent = create_agent(
        model=llm,
        tools=[save_resolution_to_db, current_time],
        system_prompt = """
                        You are a strategic planning agent. Your job is to turn a user’s New Year’s resolution into a structured calendar plan.
                        
                        Tool use (required):
                        - ALWAYS call the `current_time` tool at the start to get today's date/time (use timezone Asia/Kolkata unless the user specifies another).
                        - Use that date as the anchor for all dates you generate (e.g., “Week 1 starts on …”).
                        
                        Process:
                        1) Read the resolution and infer a realistic target timeframe if the user does not provide one (default: 4 weeks).
                        2) Create a structured calendar plan broken into weeks and days with clear milestones and measurable actions.
                        3) Include start_date (today) and end_date (based on timeframe) in the plan.
                        4) Keep the plan realistic for an average person and avoid unsafe guidance.
                        
                        Persistence:
                        - First, save the resolution and a brief summary of the plan using the provided saving tool.
                        - Then return the full structured calendar plan.
                        
                        Output rules:
                        - Return only the structured calendar plan (no extra commentary).
                        - Be specific: include dates, weekly milestones, and daily/recurring tasks.
                        """,
        response_format=CalendarPlan  # Ensures the agent returns the exact Pydantic schema
        
    )

    # Execution using the v1 invoke pattern
    # The 'messages' key is the standard input for v1 agents
    result = agent.invoke({
        "messages": [
            {"role": "user", "content": f"My resolution is: {user_resolution}"}
        ]},
        config={"callbacks": [opik_tracer]}
                         )

    return result["structured_response"]

In [77]:
# --- 4. Main Execution ---
if __name__ == "__main__":
    goal = "I want to become proficient in Python for AI development by 31 december"
    # goal = "I want to learn Python basics by tomorrow."
    print(f"Plan for: {goal}\n")
    resolution_feasibility = check_resolution_feasibility(goal)
    if resolution_feasibility.feasible:
        final_plan = run_resolution_agent(goal)
        
        print("--- Monthly Milestones ---")
        for m in final_plan.monthly_milestones:
            print(f"[{m.period}] {m.task} - Focus: {m.focus}")
    else:
        print(resolution_feasibility.reason)

    # print("\n--- Weekly Breakdown (Month 1) ---")
    # for w in final_plan.weekly_breakdown:
    #     print(f"[{w.period}] {w.task} - Focus: {w.focus}")
    
    # print("\n--- Daily Habits ---")
    # for habit in final_plan.daily_habits:


Plan for: I want to become proficient in Python for AI development by 31 december



OPIK: Filtering large LangGraph output (23729 chars) for thread display
OPIK: Filtering large LangGraph output (27584 chars) for thread display


--- Monthly Milestones ---
[1] Month 1 Milestone: Establish Python fundamentals and environment; plan starts on 2026-01-21 and ends 2026-12-31. By the end of Month 1 you will be comfortable with syntax, basic data types, and writing small scripts. - Focus: Python fundamentals and environment setup
[2] Month 2 Milestone: Master core Python features—control flow, functions, and modules; build small utilities. - Focus: Core Python features and tooling
[3] Month 3 Milestone: Deepen understanding of data structures and Pythonic patterns; begin using the standard library for small tasks. - Focus: Data structures and Pythonic patterns
[4] Month 4 Milestone: Introduction to NumPy basics and plotting with Matplotlib for data exploration. - Focus: Data science core stack introduction
[5] Month 5 Milestone: Pandas data manipulation and cleaning workflows; handle real datasets. - Focus: Data wrangling with Pandas
[6] Month 6 Milestone: Advanced data visualization and basic statistics; sharpen data

In [45]:
final_plan.weekly_breakdown

{1: [Milestone(period=1, task='Complete Python syntax exercises and write small programs.', focus='Master Python basics.'),
  Milestone(period=1, task='Practice debugging and error handling.', focus='Improve problem-solving skills.')],
 2: [Milestone(period=2, task='Learn NumPy and Pandas for data manipulation.', focus='Understand data structures and operations.'),
  Milestone(period=2, task='Create visualizations using Matplotlib.', focus='Visualize data effectively.')],
 3: [Milestone(period=3, task='Study Scikit-learn for machine learning basics.', focus='Learn model training and evaluation.'),
  Milestone(period=3, task='Experiment with TensorFlow for neural networks.', focus='Understand deep learning concepts.')],
 4: [Milestone(period=4, task='Start and complete a capstone AI project.', focus='Apply all learned skills to a real project.'),
  Milestone(period=4, task='Document the project and share it on GitHub.', focus='Showcase your work and get feedback.')]}

In [45]:
final_plan.weekly_breakdown

{1: [Milestone(period=1, task='Complete Python syntax exercises and write small programs.', focus='Master Python basics.'),
  Milestone(period=1, task='Practice debugging and error handling.', focus='Improve problem-solving skills.')],
 2: [Milestone(period=2, task='Learn NumPy and Pandas for data manipulation.', focus='Understand data structures and operations.'),
  Milestone(period=2, task='Create visualizations using Matplotlib.', focus='Visualize data effectively.')],
 3: [Milestone(period=3, task='Study Scikit-learn for machine learning basics.', focus='Learn model training and evaluation.'),
  Milestone(period=3, task='Experiment with TensorFlow for neural networks.', focus='Understand deep learning concepts.')],
 4: [Milestone(period=4, task='Start and complete a capstone AI project.', focus='Apply all learned skills to a real project.'),
  Milestone(period=4, task='Document the project and share it on GitHub.', focus='Showcase your work and get feedback.')]}